In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

import umap
from sklearn.cluster import KMeans

vitals   = pd.read_csv("../data/vitals.csv")
events   = pd.read_csv("../data/events.csv")
patients = pd.read_csv("../data/patients.csv")

def plot_patient_timeline(pid, span_hours=12, start_minute=0):
    df = vitals[vitals.patient_id==pid].copy()
    if df.empty:
        print("No such patient.")
        return
    df = df.sort_values("minute").reset_index(drop=True)

    # Slice a window
    end_minute = start_minute + span_hours*60
    dfw = df[(df.minute>=start_minute) & (df.minute<end_minute)].copy()

    ev = events[events.patient_id==pid]
    hypo_mins = set(ev[ev.event=="hypotension_start"]["minute"].tolist())

    fig, axes = plt.subplots(5, 1, figsize=(12, 9), sharex=True)
    chs = ["HR", "MAP", "SpO2", "RR", "Temp"]
    for ax, ch in zip(axes, chs):
        ax.plot(dfw["minute"], dfw[ch], lw=1.2)
        ax.set_ylabel(ch)
        # mark hypotension starts
        for m in sorted(hypo_mins):
            if start_minute <= m < end_minute:
                ax.axvline(m, ls="--", alpha=0.3)
    axes[-1].set_xlabel("minute")
    fig.suptitle(f"Patient {pid} timeline ({span_hours}h from minute {start_minute})")
    plt.tight_layout()
    plt.show()

# Example: visualize the first patient found
example_pid = int(vitals.patient_id.iloc[0])
plot_patient_timeline(example_pid, span_hours=12, start_minute=0)

In [ ]:
CTX = 120     # minutes of context
HORIZ = 60    # minutes to look ahead for event
STEP = 10     # stride between windows

SIGNALS = ["HR", "MAP", "SpO2", "RR", "Temp"]

# Build a fast lookup for hypotension start minutes per patient
hypo_dict = (events[events.event=="hypotension_start"]
             .groupby("patient_id")["minute"].apply(set)
             .to_dict())

def make_windows_for_patient(df_pid, ctx=CTX, horiz=HORIZ, step=STEP):
    """Return list of dicts with X (DataFrame window), y (0/1), metadata."""
    rows = []
    minutes = df_pid["minute"].values
    last_start = minutes.max() - (ctx + horiz)  # ensure we have enough future
    hypo_minutes = hypo_dict.get(int(df_pid.patient_id.iloc[0]), set())

    for t0 in range(0, int(last_start)+1, step):
        t1 = t0 + ctx
        t_future0, t_future1 = t1, t1 + horiz

        win = df_pid[(df_pid.minute>=t0) & (df_pid.minute<t1)]
        if len(win) < ctx * 0.9:  # too many gaps
            continue

        # label = 1 if any hypotension start in [t1, t1+horiz)
        y = int(any((m >= t_future0) and (m < t_future1) for m in hypo_minutes))

        rows.append({
            "patient_id": int(df_pid.patient_id.iloc[0]),
            "t0": t0,
            "t1": t1,
            "X": win[SIGNALS + [s+"_obs" for s in SIGNALS]].copy(),  # include obs masks
            "y": y
        })
    return rows

# Build windows for all patients (this can take a bit—okay for ~60 patients)
windows = []
for pid, dfp in vitals.groupby("patient_id"):
    dfp = dfp.sort_values("minute").reset_index(drop=True)
    windows.extend(make_windows_for_patient(dfp))

len(windows)

In [ ]:
def summarize_window_features(Xdf):
    # Xdf has columns: HR, MAP, SpO2, RR, Temp, and *_obs
    feats = {}
    for ch in SIGNALS:
        x = Xdf[ch].values.astype(float)
        obs = Xdf.get(ch+"_obs", pd.Series(np.ones_like(x))).values.astype(float)
        # basic stats
        feats[f"{ch}_mean"] = np.nanmean(x)
        feats[f"{ch}_std"]  = np.nanstd(x)
        feats[f"{ch}_min"]  = np.nanmin(x)
        feats[f"{ch}_max"]  = np.nanmax(x)
        q25, q75 = np.nanpercentile(x, [25, 75])
        feats[f"{ch}_iqr"]  = q75 - q25
        feats[f"{ch}_last"] = x[-1]
        # simple slope via linear fit on index
        idx = np.arange(len(x))
        # handle NaNs (mask)
        mask = ~np.isnan(x)
        if mask.sum() >= 5:
            a = np.polyfit(idx[mask], x[mask], 1)[0]
        else:
            a = 0.0
        feats[f"{ch}_slope"] = a
        # recent change (last - median of first half)
        mid = len(x)//2
        feats[f"{ch}_delta_mid"] = x[-1] - np.nanmedian(x[:mid])
        # missingness
        feats[f"{ch}_miss_rate"] = 1.0 - np.nanmean(obs) if obs.size else np.nan
    return feats

# Vectorize to a DataFrame
feat_rows, y_rows, pid_rows = [], [], []
for w in windows:
    feat_rows.append(summarize_window_features(w["X"]))
    y_rows.append(w["y"])
    pid_rows.append(w["patient_id"])

X = pd.DataFrame(feat_rows)
y = np.array(y_rows).astype(int)
groups = np.array(pid_rows)  # to split by patient

X.shape, y.mean()


In [ ]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
train_idx, val_idx = next(gss.split(X, y, groups))

X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
y_train, y_val = y[train_idx], y[val_idx]
groups_train = groups[train_idx]; groups_val = groups[val_idx]

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_val_sc   = scaler.transform(X_val)

# handle imbalance (if any)
pos_rate = y_train.mean()
print("Train positives:", y_train.sum(), "of", len(y_train), f"({pos_rate:.2%})")

In [ ]:
clf = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    eval_metric="logloss",
    random_state=42
)
clf.fit(X_train_sc, y_train)

p_val = clf.predict_proba(X_val_sc)[:,1]
auroc  = roc_auc_score(y_val, p_val)
auprc  = average_precision_score(y_val, p_val)

print(f"Validation AUROC: {auroc:.3f}  |  AUPRC: {auprc:.3f}")

In [ ]:
prec, rec, thr = precision_recall_curve(y_val, p_val)

plt.figure(figsize=(6,4))
plt.plot(rec, prec, lw=2)
plt.xlabel("Recall (Sensitivity)")
plt.ylabel("Precision (PPV)")
plt.title("Precision–Recall curve (validation)")
plt.grid(alpha=0.3)
plt.show()

# choose a threshold that gives ~0.3 alerts per hour per patient (demo heuristic)
# here we just set a threshold by percentile:
thr_choice = np.percentile(p_val, 100*(1-0.10))  # top 10% risk = alert
y_hat = (p_val >= thr_choice).astype(int)
cm = confusion_matrix(y_val, y_hat)
cm, thr_choice

In [ ]:
# Build a map from patient -> earliest validation hypotension start
val_pids = set(groups_val.tolist())
pid_to_first_event = {}
for pid in val_pids:
    mins = sorted(list(hypo_dict.get(int(pid), set())))
    pid_to_first_event[pid] = mins[0] if mins else None

# Gather window meta for validation set
val_meta = pd.DataFrame({
    "pid": groups_val,
    "risk": p_val,
    "y": y_val
})
# We need the corresponding t1 (window end) for each validation window
# Rebuild metadata list aligned to val_idx
t1_all = np.array([w["t1"] for w in windows])
val_t1 = t1_all[val_idx]
val_meta["t1"] = val_t1

# Choose threshold as above
val_meta["alert"] = (val_meta["risk"] >= thr_choice).astype(int)

lead_times = []
for pid, dfp in val_meta.groupby("pid"):
    event_min = pid_to_first_event.get(pid)
    if event_min is None:
        continue
    alerts_before_event = dfp[dfp["alert"]==1].copy()
    alerts_before_event = alerts_before_event[alerts_before_event["t1"] <= event_min]
    if len(alerts_before_event):
        first_alert_t1 = alerts_before_event.sort_values("t1").iloc[0]["t1"]
        lead_times.append(event_min - first_alert_t1)

if lead_times:
    lt = np.array(lead_times)
    print(f"Lead-time (minutes): median={np.median(lt):.1f}, IQR=({np.percentile(lt,25):.1f}, {np.percentile(lt,75):.1f}), n={len(lt)}")
else:
    print("No events or no alerts before events in validation.")

In [ ]:
# XGBoost feature importance (gain-based)
imp = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
imp.head(15)
plt.figure(figsize=(6,6))
imp.head(20).iloc[::-1].plot(kind="barh")
plt.title("Top 20 feature importances (XGBoost)")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer

# 1) Replace inf with NaN (just in case)
X_num = X.replace([np.inf, -np.inf], np.nan).copy()

# 2) Drop columns that are entirely NaN (can happen if a feature couldn’t be computed anywhere)
all_nan_cols = [c for c in X_num.columns if X_num[c].isna().all()]
if all_nan_cols:
    print("Dropping all-NaN columns:", all_nan_cols)
    X_num = X_num.drop(columns=all_nan_cols)

# 3) Median-impute remaining NaNs (robust for skewed clinical features)
imputer = SimpleImputer(strategy="median")
X_imp = pd.DataFrame(imputer.fit_transform(X_num), columns=X_num.columns, index=X_num.index)

# 4) Scale and embed
Z = StandardScaler().fit_transform(X_imp)
reducer = umap.UMAP(n_neighbors=30, min_dist=0.1, random_state=42)
emb = reducer.fit_transform(Z)

# now continue with KMeans + plots
k = 5
km = KMeans(n_clusters=k, random_state=42)
clusters = km.fit_predict(emb)

In [ ]:
plt.figure(figsize=(7,6))
for c in range(k):
    idx = clusters==c
    plt.scatter(emb[idx,0], emb[idx,1], s=6, alpha=0.6, label=f"C{c} (n={idx.sum()})")
plt.legend()
plt.title("UMAP embedding of windows (colored by cluster)")
plt.xlabel("UMAP-1"); plt.ylabel("UMAP-2")
plt.tight_layout()
plt.show()

# Risk-colored view (continuous)
plt.figure(figsize=(7,6))
sc = plt.scatter(emb[:,0], emb[:,1], s=6, c=y, alpha=0.7, cmap="viridis")
plt.colorbar(sc, label="Label (event within horizon)")
plt.title("UMAP embedding colored by label (risk proxy)")
plt.tight_layout()
plt.show()